In [11]:
import cv2
import face_recognition
import os
import numpy as np

known_faces_dir = 'Images'
known_face_encodings = []
known_face_names = []

for filename in os.listdir(known_faces_dir):
    if not filename.startswith('.'):  
        img = cv2.imread(os.path.join(known_faces_dir, filename))
        if img is not None:
            face_encodings = face_recognition.face_encodings(img)
            if face_encodings:
                known_face_encodings.append(face_encodings[0])
                name = os.path.splitext(filename)[0]
                known_face_names.append(name)
            else:
                print(f"No face detected in image: {filename}")
        else:
            print(f"Error loading image: {filename}")


def Eye_blinking(frame, face_location):
    top, right, bottom, left = face_location
    face_roi = frame[top:bottom, left:right]
    gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return len(eyes) >= 2

def Opening_mouth(face_landmarks, prev_open_mouth):
    top_lip_bottom = min(face_landmarks['top_lip'], key=lambda x: x[1])[1]
    bottom_lip_top = max(face_landmarks['bottom_lip'], key=lambda x: x[1])[1]
    mouth_height = bottom_lip_top - top_lip_bottom
    threshold = 60
    return prev_open_mouth or mouth_height > threshold

video_capture = cv2.VideoCapture(0)

blink_count = 0
instruction_text = "Please blink 3 times and open your mouth for liveness detection..."
open_mouth = False 
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Error: Failed to capture frame from webcam.")
        break
    cv2.putText(frame, instruction_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    face_landmarks_list = face_recognition.face_landmarks(frame, face_locations)

    for face_encoding, face_location, landmarks in zip(face_encodings, face_locations, face_landmarks_list):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distance = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distance)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            open_mouth = Opening_mouth(landmarks, open_mouth)
            eye_blink = Eye_blinking(frame, face_location)
            
            if open_mouth:
                cv2.putText(frame, "Mouth: Open", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Mouth: Closed", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            for (x, y) in landmarks['top_lip']:
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)
            for (x, y) in landmarks['bottom_lip']:
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)
        
            cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_COMPLEX
            if blink_count >= 3 and open_mouth:
                cv2.putText(frame, f"{name} (Real)", (face_location[3] + 6, face_location[2] - 6), font, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"{name} (Spoof)", (face_location[3] + 6, face_location[2] - 6), font, 1, (0, 0, 255), 2)

            if not eye_blink:
                blink_count += 1

    cv2.imshow('Face Recognition', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

Error: Failed to capture frame from webcam.
